In [1]:
import AdventOfCode

In [2]:
input <- dayLines 25

In [3]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.Attoparsec.ByteString.Char8 as A

parseLine = do
    a <- A.signed A.decimal
    A.string ","
    b <- A.signed A.decimal
    A.string ","
    c <- A.signed A.decimal
    A.string ","
    d <- A.signed A.decimal
    return (a,b,c,d)

points = map (parsed parseLine) input

In [4]:
import Data.List
import qualified Data.Map.Strict as Map
import qualified Data.Set as Set

distance (a1,b1,c1,d1) (a2,b2,c2,d2) = abs (a1-a2) + abs (b1-b2) + abs (c1-c2) + abs (d1-d2)

near p q = distance p q <= 3


initial = Map.fromList $ map (\p -> (p, Set.fromList $ filter (\q -> near p q && p /= q) points)) points

In [5]:
type GroupingMap = Map.Map (Int, Int, Int, Int) (Set.Set (Int, Int, Int, Int))

collect :: GroupingMap -> (Maybe ((Int,Int,Int,Int), Set.Set (Int, Int, Int, Int)), GroupingMap)
collect grouping | Map.null grouping = (Nothing, grouping)
collect grouping = let
    ((repr, close), rest) = Map.deleteFindMin grouping
    close' = Set.delete repr close
    nearby = Map.restrictKeys rest close'
    in case nearby of
        _ | Map.null nearby -> (Just (repr, close'), rest)
          | otherwise -> let
              elems = Map.elems nearby
              new = Set.unions (close':elems)
              other = Map.withoutKeys rest (Map.keysSet nearby)
              grouping' = Map.insert repr new other
              in collect grouping'

In [6]:
loop gs = case collect gs of
    (Nothing, _) -> []
    (Just (p,ps), rest) -> (p,ps):loop rest

length $ loop initial

396